In [1]:
from nnModel import getModel
model = getModel("gz_dev.model")

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


In [2]:
import re
import shutil
import os
import time
from os.path import isfile, join
from subprocess import Popen

currentDir = '.'
gameSourceDir = "D:/GalconZero/Games/"
gameArchiveDirPrefix = "D:/GalconZero/OldGames/Iter"
modelArchiveDir = "D:/GalconZero/OldModels"

NUM_FILES_PER_ITERATION = 4

botProcess = None

def restartBots():
    botBasePath = r"C:\Users\Evan Sparano\Documents\GitHub\galcon-alphazero\gbots"
    botProcess = Popen(botBasePath + "\launchDevBots.bat", cwd=botBasePath)

def killBots():
    if botProcess != None:
        os.kill(botProcess.pid, signal.SIGTERM)
    
def archiveGames(iterationNum):
    archiveDir = gameArchiveDirPrefix + str(iterationNum)
    os.makedirs(archiveDir)

    files = os.listdir(gameSourceDir)
    for f in files:
            shutil.move(gameSourceDir+f, archiveDir)

def getCurrentIteration():
    onlyFiles = [f for f in os.listdir(modelArchiveDir) if isfile(join(modelArchiveDir, f)) and 'iter' in f]
    return max([ int(re.findall(r'\d+', filename)[0]) for filename in onlyFiles], default=-1) + 1

def trainModel():
    time.sleep(20)
    print("trained the model!")

def nextIteration():
    # TODO: this isn't actually killing the bots, so the bots keep generating unused games while model is training
    killBots()
    
    prevIterNum = getCurrentIteration()
    model.save(modelArchiveDir + '/gz_dev.iter{0}.model'.format(prevIterNum))
    
    trainModel()
    model.save("gz_dev.model")
    
    archiveGames(prevIterNum)
    
    restartBots()
    
def startTrainingLoop(maxIterations=3):
    restartBots()
    
    while getCurrentIteration() < maxIterations:
        time.sleep(1)
        files = os.listdir(gameSourceDir)
        if len(files) >= NUM_FILES_PER_ITERATION:
            nextIteration()
            
    killBots()
    

In [3]:
startTrainingLoop()

trained the model!
trained the model!
trained the model!
